# RFP: Maze Solvers

## Project Overview
You are invited to submit a proposal that answers the following question:

### What path will your elf take?

*Please submit your proposal by **2/11/25 at 11:59 PM**.*

## Required Proposal Components

### 1. Data Description
In the code cell below, use [Gymnasium](https://gymnasium.farama.org/) to set up a [Frozen Lake maze](https://gymnasium.farama.org/environments/toy_text/frozen_lake/) for your project. When you are done with the set up, describe the reward system you plan on using.

*Note, a level 5 maze is at least 10 x 10 cells large and contains at least five lake cells.*

In [53]:
import gymnasium as gym
import random
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np

def save_pickle(file, value):
    pickle.dump(value, open(file, "wb"))
def load_pickle(file):
    return pickle.load(open(file, "rb"))

In [54]:
# Q-table this is basiclly the instruction set for the "goodness" of each move in each square


def generateMaze(size, lakes):
    ## GENERATE MAZE
    maze = []
    mazePart = []
    for x in range(lakes):
        mazePart.append("H")
    for x in range((size[0] * size[1]) - lakes - 2):
        mazePart.append("F")

    for x in range(10):
        random.shuffle(mazePart)
    ## FINALIZE MAZE
    mazePart.insert(0, "S")
    mazePart.append("G")

    ## MAKE MAZE
    count = 0
    for y in range(size[1]):
        mazeListPart = ""
        for x in range(size[0]):
            mazeListPart += mazePart[count]
            count += 1
        maze.append(mazeListPart)

    ## MAKE Q TABLE
    qTable = []
    for x in range(size[0] * size[1]):
        qTable.append([0,0,0,0])

    return maze, qTable



maze, qTable = generateMaze([10, 10], 10)


mazeStr = ""
for x in maze:
    mazeStr += x

save_pickle("maze.p", maze)
save_pickle("qTable.p", qTable)
save_pickle("mazeString.p", mazeStr)


In [55]:
maze = load_pickle("maze.p")
qTable = load_pickle("qTable.p")
mazeStr = load_pickle("mazeString.p")

# Make maze
env = gym.make('FrozenLake-v1', desc=maze, is_slippery=False, max_episode_steps=100)
initial_state = env.reset()

env.render()

# Take a step (0: LEFT, 1: DOWN, 2: RIGHT, 3: UP)
#action = 2
#new_state, reward, terminated, truncated, info = env.step(action)

env.render()


/usr/lib/python3.13/site-packages/gymnasium/envs/toy_text/frozen_lake.py:329: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("FrozenLake-v1", render_mode="rgb_array")
  gym.logger.warn(


#### Describe your reward system here.

### 2. Training Your Model
In the cell seen below, write the code you need to train a Q-Learning model. Display your final Q-table once you are done training your model.

*Note, level 5 work uses only the standard Python library and Pandas to train your Q-Learning model. A level 4 uses external libraries like Baseline3.*

In [56]:
# This function uses the Bellman Equation to update the q-table: 
    # new_q = (1-alpha) * q(s, a) + alpha * (R + gamma(max(q(s`, a`))))
def updateQTable(q, alpha, gamma, current_state, next_state, action):
    current_q = q[current_state][action]
    reward = getReward(next_state)
    next_state_row = [q[next_state][0], q[next_state][1], q[next_state][2], q[next_state][3]] 
    next_max_q = max(next_state_row)
    #print(current_q, reward, next_state_row, next_max_q)
    new_q = ((1-alpha) * current_q) + (alpha * (reward + (gamma * next_max_q)))
    q[current_state][action] = new_q
    
def getReward(state):
    if mazeStr[state] == "S" or mazeStr[state] == "F":
        return -1
    elif mazeStr[state] == "H":
        return -100
    elif mazeStr[state] == "G":
        print("GOAL")
        return 100

In [57]:
current_state = 0
alpha = 0.2 # Exploration
gamma = 0.8 # Using Knowlage
terminated = False
for x in range(10000): #Episodes
    while not terminated:
        action = random.randint(0, 3)
        new_state, reward, terminated, truncated, info = env.step(action)
        updateQTable(qTable, alpha, gamma, current_state, new_state, action)
        current_state = new_state
    inital = env.reset()
    terminated = False
env.close()
display(pd.DataFrame(qTable))
save_pickle("qTable.p", qTable)

GOAL
GOAL
GOAL
GOAL
GOAL
GOAL
GOAL
GOAL
GOAL
GOAL
GOAL
GOAL
GOAL
GOAL
GOAL
GOAL
GOAL
GOAL
GOAL
GOAL
GOAL
GOAL
GOAL


,0,1,2,3
0,-3.958492,-3.696358,-3.696589,-3.958305
1,-3.958358,-3.368459,-3.372117,-3.698308
2,-3.701384,-2.956358,-2.962461,-3.374241
3,-3.376901,-2.439764,-2.521221,-2.959622
4,-2.970441,-1.885046,-103.008232,-2.537224
...,...,...,...,...
95,-4.114461,-4.174642,-4.040690,-4.137559
96,-103.299527,28.814417,44.266041,17.253831
97,26.831390,44.934106,64.424418,34.860198
98,21.434433,53.147731,91.680977,31.594922


### 3. Testing Your Model
In the cell seen below, write the code you need to test your Q-Learning model for **1000 episodes**. It is important to test your model for 1000 episodes so that we are all able to compare our results.

*Note, level 5 testing uses both a success rate and an average steps taken metric to evaluate your model. Level 4 uses one or the other.*

In [63]:
# Test model here.

qTable = load_pickle("qTable.p")
# Make maze
env = gym.make('FrozenLake-v1', desc=maze, render_mode='rgb_array', is_slippery=False)
initial_state = env.reset()

env.render()

# Take a step (0: LEFT, 1: DOWN, 2: RIGHT, 3: UP)
#action = 2
#new_state, reward, terminated, truncated, info = env.step(action)

env.render()

def findKey(list, value):
    for keys in range(len(list)):
        if list[keys] == value:
            return keys

count = 0
current_state = 0
terminated = False
while not terminated:
    action = findKey(qTable[current_state], max(qTable[current_state]))
    print(action)
    new_state, reward, terminated, truncated, info = env.step(action)
    current_state = new_state
    
    img = env.render()
    plt.imshow(img)
    plt.axis('off')  # Turn off axis
    plt.savefig('images/frame' + str(count) + '.png', bbox_inches='tight', pad_inches=0)
    plt.close()
    count += 1
env.close()


1
2
2
2
1
2
1
1
1
1
1
2
2
2
1
2
2
1


In [61]:
env.close()

### 4. Final Answer
In the first cell below, describe the path your elf takes to get to the gift. *Note, a level 5 answer includes a gif of the path your elf takes in order to reach the gift.*

In the second cell seen below, describe how well your Q-Learning model performed. Make sure that you explicitly name the **learning rate**, **the discount factor**, and the **reward system** that you used when training your final model. *Note, a level 5 description describes the model's performance using two types of quantitative evidence.*

![example image](final.gif)

#### Describe the path your elf takes here.

The elf takes an almost straight across path to the gift avoiding all lakes

#### Describe how well your Q-Learning model performed here.

It preformed relativly well if the weghts were ajusted so that a single tile is more punishing then it could be taught to find the shortest possible path. It would also help the algorithm if the directions were more random.

